In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Flatten
from enum import Enum

class Symbols(Enum):
    BLANK = " "
    ALGIZ = "ᛉ"
    ANSUZ = "ᚨ"
    BERKANO = "ᛒ"
    DAGAZ = "ᛞ"
    EHWAZ = "ᛖ"
    EIWAZ = "ᛇ"
    FEHU = "ᚠ"
    GEBO = "ᚷ"
    HAGALAZ = "ᚺ"
    INGWAZ = "ᛜ"
    ISA = "ᛁ"
    JERA = "ᛃ"
    KENAZ = "ᚲ"
    LAGUZ = "ᛚ"
    MANNAZ = "ᛗ"
    NAUDIZ = "ᚾ"
    OTHALA = "ᛟ"
    PERTHO = "ᛈ"
    RAIDHO = "ᚱ"
    SOWILO = "ᛋ"
    THURISAZ = "ᚦ"
    TIWAZ = "ᛏ"
    URAZ = "ᚢ"
    WUNJO = "ᚹ"
    GARBAGE = ""

def load_images(data_dir):
    images = []
    labels = []
    for symbol in Symbols:
        symbol_dir = os.path.join(data_dir, symbol.value)
        for filename in os.listdir(symbol_dir):
            if filename.endswith('.png'):
                img_path = os.path.join(symbol_dir, filename)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                img = cv2.resize(img, (28, 28))
                images.append(img)
                labels.append(symbol.value)
    images = np.array(images)
    labels = np.array(labels)
    return images, labels

def build_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(len(Symbols), activation='softmax'))
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

def main():
    data_dir = 'path/to/dataset'
    images, labels = load_images(data_dir)
    images = images.reshape(images.shape[0], 28, 28, 1)
    labels = [Symbols[l] for l in labels]
    labels = [l.value for l in labels]
    labels = to_categorical(labels)
    X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.33)
    model = build_model()
    model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))
    test_loss, test_acc = model.evaluate(X_test, y_test)
    print('Test accuracy:', test_acc)
    text = "INSERT LIBER PRIMUS
